### New Approach to Merging  
Uses last quarter 2022 data from both ASMT and TRANS.  
See documentation summary titled "ZTRAX Methodology."

In [1]:
import pandas as pd
import csv

### Step 1: Left join ASMT Main to ASMT Building on RowID into a dataframe called asmt_merge.

Read in the ASMT Main file.

In [3]:
asmt_main = pd.read_csv(
        '../20221103_GA/ZAsmt/Main.txt',
        sep='|',
        on_bad_lines='skip',
        encoding='latin-1',
        quoting=csv.QUOTE_NONE,
        header=None,
        usecols=[0, 1, 2, 4, 20, 26, 27, 29, 30, 35, 36, 38, 39, 40, 70, 92],
        names=["RowID", "ImportParcelID", "FIPS", "County", "PropertyHouseNumber",
                "PropertyFullStreetAddress", "PropertyCity", "PropertyZip", "PropertyZip4",
                "PropertyZoningSourceCode", "CensusTract", "TaxAmount", "TaxYear",
                "TaxDelinquencyFlag", "LotSizeSquareFeet", "BatchID"],
        dtype={"RowID": pd.StringDtype(), "ImportParcelID": "Int32", "FIPS": "Int16",
               "County": "category", "PropertyHouseNumber": pd.StringDtype(), "PropertyFullStreetAddress": pd.StringDtype(),
               "PropertyCity": "category", "PropertyZip": "category", "PropertyZip4": "category",
               "PropertyZoningSourceCode": "category", "CensusTract": "Int32",
               "TaxAmount": "float64", "TaxYear": "Int16", "TaxDelinquencyFlag": "category",
               "LotSizeSquareFeet": "float64", "BatchID": "Int32"})

Print sample of ASMT Main to verify it was read in correctly.

In [4]:
asmt_main.head(3)

,RowID,ImportParcelID,FIPS,County,PropertyHouseNumber,PropertyFullStreetAddress,PropertyCity,PropertyZip,PropertyZip4,PropertyZoningSourceCode,CensusTract,TaxAmount,TaxYear,TaxDelinquencyFlag,LotSizeSquareFeet,BatchID
0,05A32702-0000-0000-32C9-000000000000,36152069,13001,Appling,<NA>,<NA>,NaN,NaN,NaN,NaN,<NA>,154.91,2021,NaN,131986.8,1002195114
1,06A32702-0000-0000-32C9-000000000000,36152070,13001,Appling,<NA>,<NA>,NaN,NaN,NaN,NaN,<NA>,18.26,2021,NaN,17859.6,1002195114
2,07A32702-0000-0000-32C9-000000000000,36152071,13001,Appling,<NA>,<NA>,NaN,NaN,NaN,NaN,<NA>,601.74,2021,NaN,87120.0,1002195114


Counting duplicates of RowID and ImportParcelID in ASMT Main (there should not be any).

In [5]:
print("Duplicates of RowID: ", asmt_main['RowID'].duplicated().sum())
print("Duplicates of ImportParcelID: ", asmt_main['ImportParcelID'].duplicated().sum())

Duplicates of RowID:  0
Duplicates of ImportParcelID:  0


Select only rows that are in the 29-county Atlanta–Sandy Springs–Alpharetta MSA.

In [6]:
FIPS_GA = [13013, 13015, 13035, 13045, 13057, 13063, 13067, 13077, 13085, 13089,
           13089, 13097, 13113, 13117, 13121, 13135, 13143, 13149, 13151, 13159,
           13171, 13199, 13211, 13217, 13223, 13227, 13231, 13247, 13255, 13297]

asmt_main = asmt_main.loc[asmt_main['FIPS'].isin(FIPS_GA)]

Size after selecting only properties in Atlanta MSA.

In [7]:
asmt_main.shape

(2240380, 16)

Result: There are 2,240,380 assessed parcels in the Atlanta MSA according to ASMT Main data.

Read in the ASMT Building file.

In [15]:
asmt_building = pd.read_csv(
    '../20221103_GA/ZAsmt/Building.txt',
    sep='|',
    on_bad_lines='skip',
    encoding='latin-1',
    quoting=csv.QUOTE_NONE,
    header=None,
    usecols=[0, 1, 4, 5, 14, 17, 18, 19, 25, 44],
    names=["RowID", "NoOfUnits", "PropertyCountyLandUseCode", "PropertyLandUseStndCode",
           "YearBuilt", "NoOfStories", "TotalRooms", "TotalBedrooms", "TotalCalculatedBathCount", "StoryTypeStndCode"],
    dtype={"RowID": pd.StringDtype(), "NoOfUnits": "Int32",
           "PropertyCountyLandUseCode": "category", "PropertyLandUseStndCode": "category", "NoOfStories": "float32",
           "PropertyStateLandUseCode": "category", "YearBuilt": "Int16", "TotalRooms": "Int16",
           "TotalBedrooms": "category", "TotalCalculatedBathCount": "float32",
           "StoryTypeStndCode": "category"})

Print sample of ASMT Building to verify it was read in correctly.

In [16]:
asmt_building.head(3)

,RowID,NoOfUnits,PropertyCountyLandUseCode,PropertyLandUseStndCode,YearBuilt,NoOfStories,TotalRooms,TotalBedrooms,TotalCalculatedBathCount,StoryTypeStndCode
0,05A32702-0000-0000-32C9-000000000000,<NA>,1,RR101,1991,1.0,<NA>,NaN,1.0,NaN
1,06A32702-0000-0000-32C9-000000000000,<NA>,R4,VL101,<NA>,NaN,<NA>,NaN,NaN,NaN
2,07A32702-0000-0000-32C9-000000000000,<NA>,1,RR101,1900,1.0,<NA>,NaN,1.0,NaN


Counting duplicates of RowID in ASMT Building.

In [17]:
print("Entire row duplicated (auto dropped): ", asmt_building.duplicated().sum())
asmt_building.drop_duplicates(inplace=True)

print("--------")
num_dup = asmt_building['RowID'].duplicated().sum()
print("Duplicates (RowID): ", num_dup)
print("Total size: ", asmt_building.shape)
print("Percent: ", num_dup / len(asmt_building.index) * 100)

Entire row duplicated (auto dropped):  89721
--------
Duplicates (RowID):  120475
Total size:  (4913967, 10)
Percent:  2.4516851659768983


Left join ASMT Main and ASMT Building on RowID.

In [18]:
asmt_merge = asmt_main.merge(asmt_building, how="left", on="RowID")
print("Main shape: ", asmt_main.shape)
print("Building shape: ", asmt_building.shape)
print("Merged shape: ", asmt_merge.shape)

Main shape:  (2240380, 16)
Building shape:  (4913967, 10)
Merged shape:  (2278953, 25)


### Sub-step 1.1
Remove rows from asmt_merge that are not Single Family Residential.

Drop non-SFH to see distribution and number of duplicates. Print number of dropped entries and current size.

In [19]:
init_len = len(asmt_merge.index)
asmt_merge = asmt_merge.loc[asmt_merge['PropertyLandUseStndCode'].isin(['RR101', 'RR999'])]
print("Dropped (non-SFH): ", init_len - len(asmt_merge.index))
print("Percent dropped: ", (init_len - len(asmt_merge.index)) / init_len * 100)
print("New total size: ", asmt_merge.shape)

Dropped (non-SFH):  669246
Percent dropped:  29.366380087698168
New total size:  (1609707, 25)


Distribution of RR101 and RR999.

In [20]:
asmt_merge['PropertyLandUseStndCode'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

RR101    96.3%
RR999     3.7%
AG199     0.0%
CR107     0.0%
CR118     0.0%
         ...  
CR130     0.0%
CR131     0.0%
CR147     0.0%
CR149     0.0%
MS109     0.0%
Name: PropertyLandUseStndCode, Length: 221, dtype: object

New percent of data which has duplicate RowID (SFH only).

In [21]:
num_dup = asmt_merge['RowID'].duplicated().sum()
print("Number of duplicates: ", num_dup)
print("Percent: ", num_dup / len(asmt_merge.index) * 100)

Number of duplicates:  10707
Percent:  0.6651521053210305


DETERMINATION: Since rows with duplicate RowID are .64% of merged data, and the distribution for duplicates of LandUseCode is similiar to the overall -> drop all duplicate rows (including the first observation with that RowID, since that parcel has multiple buildings too) for simplicity.  
First, we go back to ASMT Building and drop rows with duplicate RowIDs, keeping none (not even the first entry), then we re-do the merge. This ensures an accurate merge.

For research purposes, we are only looking at parcels with single-family parcels in the Atlanta MSA with one building on them. Parcels with more than one building on them are extremely insignificant.

In [22]:
total_size = len(asmt_merge.index)

asmt_merge = asmt_merge.drop_duplicates(subset='RowID', keep=False)
print("Dropped duplicates: ", total_size - len(asmt_merge.index))

Dropped duplicates:  20330


In [23]:
print("Percent: ", (total_size - len(asmt_merge.index)) / total_size * 100)

Percent:  1.2629627627885074


Verify no duplications in asmt_merge.

In [24]:
print("Count of entire row duplications: ", asmt_merge.duplicated().sum())
print("Count of RowID duplications: ", asmt_merge['RowID'].duplicated().sum())

Count of entire row duplications:  0
Count of RowID duplications:  0


### Step 2
Left join TRANS PropertyInfo to asmt_merge on ImportParcelID, creating a new dataframe called combined.

Read in TRANS PropertyInfo.

In [49]:
trans_property_info = pd.read_csv(
    '../20221103_GA/ZTrans/PropertyInfo.txt',
    sep='|',
    on_bad_lines='skip',
    encoding='latin-1',
    quoting=csv.QUOTE_NONE,
    header=None,
    usecols=[0, 8, 15, 16, 18, 19, 46, 62, 64],
    names=["TransId", "PropertyHouseNumber", "PropertyFullStreetAddress", "PropertyCity",
    "PropertyZip", "PropertyZip4", "PropertySequenceNumber", "FIPS", "ImportParcelID"],
    dtype={"TransId": "Int32", "PropertyHouseNumber": pd.StringDtype(),
    "PropertyFullStreetAddress": pd.StringDtype(), "PropertyCity": "category",
    "PropertyZip": "category", "PropertyZip4": "category", "FIPS": "Int16", "ImportParcelID": "Int32"})

Print sample of TRANS PropertyInfo to verify it was read in correctly.

In [50]:
trans_property_info.head(3)

,TransId,PropertyHouseNumber,PropertyFullStreetAddress,PropertyCity,PropertyZip,PropertyZip4,PropertySequenceNumber,FIPS,ImportParcelID
0,148390440,<NA>,HOLMESVILLE AVE,NaN,31513,,1,13001,36162962
1,148390441,<NA>,SURSSON ST,NaN,31513,,1,13001,36161646
2,148390442,473,473 James Kent Rd,Baxley,31513,7404,1,13001,36156578


Select only rows that are in the 29-county Atlanta–Sandy Springs–Alpharetta MSA.

In [51]:
FIPS_GA = [13013, 13015, 13035, 13045, 13057, 13063, 13067, 13077, 13085, 13089,
           13089, 13097, 13113, 13117, 13121, 13135, 13143, 13149, 13151, 13159,
           13171, 13199, 13211, 13217, 13223, 13227, 13231, 13247, 13255, 13297]

trans_property_info = trans_property_info.loc[trans_property_info['FIPS'].isin(FIPS_GA)]
print("New size: ", trans_property_info.shape)

New size:  (5466709, 9)


Examine duplicates.

In [52]:
print("Entire row duplicated (auto dropped): ", trans_property_info.duplicated().sum())

print("--------")
num_dup = trans_property_info['TransId'].duplicated().sum()
print("Duplicates (TransId): ", num_dup)
print("Total size: ", trans_property_info.shape)
print("Percent: ", num_dup / len(trans_property_info.index) * 100)

print("--------")
print("Count of duplicate ImportParcelID: ", trans_property_info['ImportParcelID'].duplicated().sum())

Entire row duplicated (auto dropped):  0
--------
Duplicates (TransId):  170164
Total size:  (5466709, 9)
Percent:  3.112731992868104
--------
Count of duplicate ImportParcelID:  4108795


Print the distribution of PropertySequenceNumber, which indexes properties involved in one transcation.

In [53]:
trans_property_info['PropertySequenceNumber'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

1      96.9%
2       0.7%
3       0.3%
4       0.2%
5       0.2%
       ...  
453     0.0%
452     0.0%
451     0.0%
450     0.0%
501     0.0%
Name: PropertySequenceNumber, Length: 501, dtype: object

DETERMINATION: Duplications of TransId in TRANS PropertyInfo occur when there are multiple properties, with different property info, involved in a single transcation.  

For research purposes, we do not want to consider transcations with more than one property involved. Not only is this an insignificant portion of the data (see exact percentage in following cells), but also these transcations are bulk sales or otherwise do not involve individual buyers (e.g. they are corporations selling to corporations).

WE are ONLY looking at transcations with 1 PROPERTY ASSOCIATED.  

---

Left join modified TRANS PropertyInfo and asmt_merge into a new dataframe called combined.  
A left join onto transcation data is preferred as our goal is to create a transcation database with assessment info included for the properties involved. Thus, transcation data should be treated as the parent/main data for us to merge onto. Further, for our research, we only care about properties that have been involved in a transcation. If the property was never sold over the time we analyize, it is not important to us.

In [77]:
combined = trans_property_info.merge(asmt_merge, how="left", on="ImportParcelID")
print("Asmt merged shape: ", asmt_merge.shape)
print("Trans shape: ", trans_property_info.shape)
print("Combined shape: ", combined.shape)

Asmt merged shape:  (1589377, 25)
Trans shape:  (5466709, 9)
Combined shape:  (5466709, 33)


Print a sample of combined to verify the merge was successful.

There are some duplicated columns, labeled "[name]_x" if from TRANS PropertyInfo, and "[name]_y" if from asmt_merge. I have decided to keep these for now and will modify them later based on which columns are more complete.

In [78]:
combined.head(3)

,TransId,PropertyHouseNumber_x,PropertyFullStreetAddress_x,PropertyCity_x,PropertyZip_x,PropertyZip4_x,PropertySequenceNumber,FIPS_x,ImportParcelID,RowID,...,BatchID,NoOfUnits,PropertyCountyLandUseCode,PropertyLandUseStndCode,YearBuilt,NoOfStories,TotalRooms,TotalBedrooms,TotalCalculatedBathCount,StoryTypeStndCode
0,148470374,286,286 Fiddlers Trl,Auburn,30011,3350,1,13013,36214022,06952802-0000-0000-32D5-000000000000,...,1002049436,<NA>,1,RR101,1988,7.0,5,3,3.0,SPL
1,148470375,1542,1542 Oakleaf Dr,Auburn,30011,3268,1,13013,36213235,F3912802-0000-0000-32D5-000000000000,...,1002049436,1,1,RR101,1990,1.0,5,3,2.0,SGL
2,148470605,<NA>,WILLIAMS RD,AUBURN,30203,,1,13013,<NA>,<NA>,...,<NA>,<NA>,NaN,NaN,<NA>,NaN,<NA>,NaN,NaN,NaN


Examine duplicates in combined.

In [79]:
print("Count of entire row duplications: ", combined.duplicated().sum())
print("Count of ImportParcelID duplications (some expected): ", combined['ImportParcelID'].duplicated().sum())
print("Count of rows with duplicate RowID (some expected): ", combined["RowID"].duplicated().sum())
print("Count of rows with duplicate TransId: ", combined["TransId"].duplicated().sum())

Count of entire row duplications:  0
Count of ImportParcelID duplications (some expected):  4108795
Count of rows with duplicate RowID (some expected):  4391366
Count of rows with duplicate TransId:  170164


Drop all duplicates of TranId in TRANS PropertyInfo, including the first observation with that TransId since this property was involved in the bulk sale too. Print the number and percent of dropped observations (e.g. the number of transcations with more than one property involved).

In [80]:
init_size = len(combined.index)

combined = combined.drop_duplicates(subset='TransId')
dup = init_size - len(combined.index)
print("Count of rows with duplicate TransId dropped: ", dup)
print("Percent dropped: ", dup / init_size * 100)

Count of rows with duplicate TransId dropped:  170164
Percent dropped:  3.112731992868104


Transcations with more than one property involved in the Atlanta MSA: 3.1% (insignificant and unimportant for research questions, so it was dropped).  

Now we have a merged dataframe, combined, containing ASMT Main & Building, and TRANS PropertyInfo.  
In combined, there are no duplicate TransIds, meaning we can accurately merge further transaction data.  
There are duplicate RowIDs, which is expected, as a property might have been involved in multiple transcations over time.

### Step 3
Left join new combined dataframe and TRANS Main.  

At first, I was worried that we first merged TRANS PropertyInfo to asmt_merged and are using that as our left/source for this join; however, TRANS PropertyInfo has about 500K more observations than TRANS Main and we also wanted to eliminate transactions with multiple properties involved, so it should be fine.

Read in TRANS Main.

In [81]:
trans_main = pd.read_csv(
    '../20221103_GA/ZTrans/Main.txt',
    sep='|',
    on_bad_lines='skip',
    encoding='latin-1',
    quoting=csv.QUOTE_NONE,
    header=None,
    usecols=[0, 1, 3, 4, 6, 16, 24, 25, 30, 32, 62, 104, 105, 127],
    names=["TransId", "FIPS", "County", "DocumentTypeStndCode", "RecordingDate", "DataClassStndCode", "SalesPriceAmount", "SalesPriceAmountStndCode", "IntraFamilyTransferFlag",
        "PropertyUseStndCode", "LoanTypeStndCode", "TotalDelinquentAmount", "DelinquentAsOfDate", "BatchID"],
    dtype={"TransId": "Int32", "FIPS": "Int32", "County": "category", "DocumentTypeStndCode": "category", "DataClassStndCode": "category", "RecordingDate": pd.StringDtype(), "SalesPriceAmount": "Float32", "SalesPriceAmountStndCode": "category", "IntraFamilyTransferFlag": "category",
        "PropertyUseStndCode": "category", "LoanTypeStndCode": "category", "TotalDelinquentAmount": "Float32", "DelinquentAsOfDate": pd.StringDtype(), "BatchID": "Int32"})

In [82]:
trans_main.head(3)

,TransId,FIPS,County,DocumentTypeStndCode,RecordingDate,DataClassStndCode,SalesPriceAmount,SalesPriceAmountStndCode,IntraFamilyTransferFlag,PropertyUseStndCode,LoanTypeStndCode,TotalDelinquentAmount,DelinquentAsOfDate,BatchID
0,148390440,13001,APPLING,D,2000-06-14,WRDE,<NA>,NO,NaN,NaN,NaN,<NA>,<NA>,1002195118
1,148390441,13001,APPLING,D,2004-03-12,WRDE,6500.0,RD,NaN,UL,NaN,<NA>,<NA>,1002195118
2,148390442,13001,APPLING,D,2004-04-25,WRDE,1500.0,RD,NaN,UL,NaN,<NA>,<NA>,1002195118


Select only rows that are in the 29-county Atlanta–Sandy Springs–Alpharetta MSA.

In [83]:
FIPS_GA = [13013, 13015, 13035, 13045, 13057, 13063, 13067, 13077, 13085, 13089,
           13089, 13097, 13113, 13117, 13121, 13135, 13143, 13149, 13151, 13159,
           13171, 13199, 13211, 13217, 13223, 13227, 13231, 13247, 13255, 13297]

trans_main = trans_main.loc[trans_main['FIPS'].isin(FIPS_GA)]
print("New size: ", trans_main.shape)

New size:  (5296545, 14)


### Step 3.1
Remove non-arms length based on DataClassStndCode.

In [84]:
non_arms = ["QCDE", "INTR", "SHDE", "AFDT", "DELU", "LFQC", "RCDE", "RDDE", "TRFC", "TXDE"]

init_size = len(trans_main.index)

trans_main = trans_main.loc[~trans_main['DataClassStndCode'].isin(non_arms)]

dropped = init_size - len(trans_main.index)
print("Count of non-arms transactions dropped: ", dropped)
print("Percent: ", dropped / init_size * 100)

Count of non-arms transactions dropped:  188831
Percent:  3.565173145890387


### Step 3.2
Remove empty RecordingDate, SalesPriceAmount, and SalesPriceAmount < 1000 from TRANS Main.

PROBLEM (?): *This eliminated a lot: ~71%*

In [85]:
init_size = len(trans_main.index)

trans_main = trans_main[trans_main['RecordingDate'].notna()]
new_size = len(trans_main.index)
print("Count dropped by eliminating empty DocumentDate: ", init_size - new_size)

trans_main = trans_main[trans_main['SalesPriceAmount'].notna()]
prev_size = new_size
new_size = len(trans_main.index)
print("Count dropped by eliminating empty SalesPriceAmount: ", prev_size - new_size)

trans_main = trans_main[trans_main['SalesPriceAmount'] != 0]
prev_size = new_size
new_size = len(trans_main.index)
print("Count dropped by eliminating SalesPriceAmount == 0: ", prev_size - new_size)

trans_main = trans_main.loc[trans_main['SalesPriceAmount'] >= 1000]
prev_size = new_size
new_size = len(trans_main.index)
print("Count dropped by eliminating SalesPriceAmount < 1000: ", prev_size - new_size)

dropped = init_size - len(trans_main.index)

print("Total dropped: ", dropped)
print("Percent: ", dropped / init_size * 100)

Count dropped by eliminating empty DocumentDate:  0
Count dropped by eliminating empty SalesPriceAmount:  3610966
Count dropped by eliminating SalesPriceAmount == 0:  0
Count dropped by eliminating SalesPriceAmount < 1000:  902
Total dropped:  3611868
Percent:  70.7139828110971


### Step 3.3
Create a dummy variable if (DataClassStndCode == "FCDE") == 1 (foreclosure deed).

Duplicate and append column, then convert it to a dummy.

In [86]:
dummy_col = trans_main.loc[:, 'DataClassStndCode'] #Select DataClassStndCode column
dummy_col = dummy_col.cat.set_categories(['FCDE', 'False'])
dummy_col = dummy_col.fillna("False")
dummy_col


123690      False
123692      False
123693      False
123694      False
123695      False
            ...  
11413017    False
11413018    False
11413019    False
11413020    False
11413021    False
Name: DataClassStndCode, Length: 1495846, dtype: category
Categories (2, object): ['FCDE', 'False']

In [87]:
dummy_col = dummy_col.map({'FCDE': 1, 'False': 0})
dummy_col

123690      0
123692      0
123693      0
123694      0
123695      0
           ..
11413017    0
11413018    0
11413019    0
11413020    0
11413021    0
Name: DataClassStndCode, Length: 1495846, dtype: category
Categories (2, int64): [1, 0]

Verify the number of columns that have a '1', meaning FCDE code, in the dummy column and in the original data (should be the same).

In [88]:
print("Number of rows with code FCDE in original data: ", len(trans_main.loc[trans_main['DataClassStndCode'] == 'FCDE'].index))
print("Number of rows with code FCDE in dummy column: ", len(dummy_col.loc[dummy_col == 1].index))

Number of rows with code FCDE in original data:  101249
Number of rows with code FCDE in dummy column:  101249


Convert the dummy column to a boolean.

In [89]:
dummy_col = dummy_col.astype('bool')
dummy_col

123690      False
123692      False
123693      False
123694      False
123695      False
            ...  
11413017    False
11413018    False
11413019    False
11413020    False
11413021    False
Name: DataClassStndCode, Length: 1495846, dtype: bool

Create a new column in TRANS Main called 'FCDE' with the dummy column.

In [90]:
trans_main['FCDE'] = dummy_col
trans_main

,TransId,FIPS,County,DocumentTypeStndCode,RecordingDate,DataClassStndCode,SalesPriceAmount,SalesPriceAmountStndCode,IntraFamilyTransferFlag,PropertyUseStndCode,LoanTypeStndCode,TotalDelinquentAmount,DelinquentAsOfDate,BatchID,FCDE
123690,148470374,13013,BARROW,D,1993-08-13,DEED,81900.0,RD,NaN,SR,NaN,<NA>,<NA>,1002049440,False
123692,148470605,13013,BARROW,D,1994-01-24,DEED,49000.0,RD,NaN,SR,NaN,<NA>,<NA>,1002049440,False
123693,148470703,13013,BARROW,D,1994-03-09,DEED,37400.0,RD,NaN,SR,NaN,<NA>,<NA>,1002049440,False
123694,148471097,13013,BARROW,D,1994-07-05,DEED,9500.0,NaN,NaN,UL,NaN,<NA>,<NA>,1002049440,False
123695,148471129,13013,BARROW,D,1994-07-14,DEED,89900.0,RD,NaN,SR,NaN,<NA>,<NA>,1002049440,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11413017,578372459,13297,Walton,D,2022-10-07,WRDE,274000.0,RD,NaN,NaN,NaN,<NA>,<NA>,1002277953,False
11413018,578372460,13297,Walton,D,2022-10-07,WRDE,570000.0,RD,NaN,NaN,NaN,<NA>,<NA>,1002277953,False
11413019,578372461,13297,Walton,D,2022-10-07,WRDE,279900.0,RD,NaN,NaN,NaN,<NA>,<NA>,1002277953,False
11413020,578372462,13297,Walton,D,2022-10-11,WRDE,1350000.0,RD,NaN,NaN,NaN,<NA>,<NA>,1002277953,False


Left join combined and the modified TRANS Main on key TransId.

In [91]:
combined = combined.merge(trans_main, how="left", on="TransId")
combined

,TransId,PropertyHouseNumber_x,PropertyFullStreetAddress_x,PropertyCity_x,PropertyZip_x,PropertyZip4_x,PropertySequenceNumber,FIPS_x,ImportParcelID,RowID,...,DataClassStndCode,SalesPriceAmount,SalesPriceAmountStndCode,IntraFamilyTransferFlag,PropertyUseStndCode,LoanTypeStndCode,TotalDelinquentAmount,DelinquentAsOfDate,BatchID_y,FCDE
0,148470374,286,286 Fiddlers Trl,Auburn,30011,3350,1,13013,36214022,06952802-0000-0000-32D5-000000000000,...,DEED,81900.0,RD,NaN,SR,NaN,<NA>,<NA>,1002049440,False
1,148470375,1542,1542 Oakleaf Dr,Auburn,30011,3268,1,13013,36213235,F3912802-0000-0000-32D5-000000000000,...,NaN,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN
2,148470605,<NA>,WILLIAMS RD,AUBURN,30203,,1,13013,<NA>,<NA>,...,DEED,49000.0,RD,NaN,SR,NaN,<NA>,<NA>,1002049440,False
3,148470703,505,505 Yargo Ln,Winder,30680,3954,1,13013,36234911,9FE62802-0000-0000-32D5-000000000000,...,DEED,37400.0,RD,NaN,SR,NaN,<NA>,<NA>,1002049440,False
4,148471097,384,384 Scenic Ln,Auburn,30011,2516,1,13013,36213478,E6922802-0000-0000-32D5-000000000000,...,DEED,9500.0,NaN,NaN,UL,NaN,<NA>,<NA>,1002049440,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5296540,578457745,5787,5787 Mason Jones Dr,Powder Springs,30127,4761,1,13067,37184519,07643702-0000-0000-330B-000000000000,...,NaN,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN
5296541,578457746,724,724 Registry Run NW,Kennesaw,30152,2870,1,13067,37229517,CD133802-0000-0000-330B-000000000000,...,NaN,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN
5296542,578457747,4834,4834 Olde Mill Dr,Marietta,30066,1159,1,13067,37000623,AF953402-0000-0000-330B-000000000000,...,NaN,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN
5296543,578457748,5138,5138 Lakeview Ct,Austell,30106,2737,1,13067,37181434,<NA>,...,NaN,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN


You will notice columns labeled "{name}_x" and "{name}_y". These are when columns with the same names were merged from separate files. We need to decide which to keep- or fill in values from both to create a more complete, singular column for that variable. This will be done later.

Examine any duplications in combined after merging TRANS Main.

In [92]:
print("Count of entire row duplications: ", combined.duplicated().sum())
print("Count of ImportParcelID duplications (some expected): ", combined['ImportParcelID'].duplicated().sum())
print("Count of rows with duplicate RowID (some expected): ", combined["RowID"].duplicated().sum())
print("Count of rows with duplicate TransId: ", combined["TransId"].duplicated().sum())

Count of entire row duplications:  0
Count of ImportParcelID duplications (some expected):  3963499
Count of rows with duplicate RowID (some expected):  4231197
Count of rows with duplicate TransId:  0


### Step 4
Left join combined and ASMT BuildingAreas. Before merging, create 'BAT' and 'BAL' columns and get rid of 'BuildingAreaStndCode' and 'BuildingAreaSqFt' columns to eliminate any duplication issues.

Read in ASMT BuildingAreas.

In [104]:
asmt_building_areas = pd.read_csv(
    '../20221103_GA/ZAsmt/BuildingAreas.txt',
    sep='|',
    on_bad_lines='skip',
    encoding='latin-1',
    quoting=csv.QUOTE_NONE,
    header=None,
    usecols=[0, 3, 4],
    names=["RowID", "BuildingAreaStndCode", "BuildingAreaSqFt"],
    dtype={"RowID": pd.StringDtype(), "BuildingAreaStndCode": "category", "BuildingAreaSqFt": "Int32"})

In [105]:
asmt_building_areas.head(3)

,RowID,BuildingAreaStndCode,BuildingAreaSqFt
0,05A32702-0000-0000-32C9-000000000000,BAH,1080
1,07A32702-0000-0000-32C9-000000000000,BAH,2836
2,0AA32702-0000-0000-32C9-000000000000,BAH,1176


Examine duplicates in TRANS BuildingAreas.

In [106]:
print("Count of entire row duplications (auto dropped): ", asmt_building_areas.duplicated().sum())
asmt_building_areas.drop_duplicates(inplace=True)

print("Count of rows with duplicate RowIDs: ", asmt_building_areas['RowID'].duplicated().sum())

Count of entire row duplications (auto dropped):  101956
Count of rows with duplicate RowIDs:  4638180


In [107]:
store_original_df = asmt_building_areas.copy()

### Sub-step: 4.1  
Create columns for BAT and BAL; drop any rows not matching BAT or BAL.

Create a copy of the dataframe and set the only acceptable data for 'BuildingAreaStndCode' as 'BAL' or 'BAT.' Other data will then be filled in as NaN.

In [108]:
select_buildingarea_codes = asmt_building_areas.copy()
select_buildingarea_codes['BuildingAreaStndCode'] = select_buildingarea_codes['BuildingAreaStndCode'].cat.set_categories(['BAL', 'BAT'])
select_buildingarea_codes

,RowID,BuildingAreaStndCode,BuildingAreaSqFt
0,05A32702-0000-0000-32C9-000000000000,NaN,1080
1,07A32702-0000-0000-32C9-000000000000,NaN,2836
2,0AA32702-0000-0000-32C9-000000000000,NaN,1176
3,0BA32702-0000-0000-32C9-000000000000,NaN,1680
4,0EA32702-0000-0000-32C9-000000000000,BAT,1068
...,...,...,...
8463503,C653E10B-0000-0000-33FF-000000000000,BAL,1580
8463504,C653E10B-0000-0000-33FF-000000000000,NaN,1580
8463505,C653E10B-0000-0000-33FF-000000000000,NaN,1580
8463506,C653E10B-0000-0000-33FF-000000000000,NaN,120


Only select rows where 'BuildingAreaStndCode' is not NaN.

In [109]:
select_buildingarea_codes = select_buildingarea_codes[select_buildingarea_codes['BuildingAreaStndCode'].notna()]
select_buildingarea_codes

,RowID,BuildingAreaStndCode,BuildingAreaSqFt
4,0EA32702-0000-0000-32C9-000000000000,BAT,1068
84,83A32702-0000-0000-32C9-000000000000,BAT,9360
86,86A32702-0000-0000-32C9-000000000000,BAT,1600
87,87A32702-0000-0000-32C9-000000000000,BAT,2220
91,8BA32702-0000-0000-32C9-000000000000,BAT,3600
...,...,...,...
8463492,BE53E10B-0000-0000-33FF-000000000000,BAL,1978
8463495,BF53E10B-0000-0000-33FF-000000000000,BAL,2794
8463498,C153E10B-0000-0000-33FF-000000000000,BAL,3748
8463501,C553E10B-0000-0000-33FF-000000000000,BAL,1584


Count of dropped rows (where 'BuildingAreaStndCode' != BAL, BAT).

In [110]:
print("Count dropped where BuildingAreaCode != BAL, BAT: ", len(asmt_building_areas.index) - len(select_buildingarea_codes.index))
asmt_building_areas = select_buildingarea_codes

Count dropped where BuildingAreaCode != BAL, BAT:  6751682


Fill in a new 'BAL' column with 'BuildingAreaSqFt' when 'BuildingAreaStndCode' == BAL; do the same for 'BAT' codes.

In [111]:
asmt_building_areas['BAL'] = (asmt_building_areas.loc[asmt_building_areas['BuildingAreaStndCode'] == 'BAL']['BuildingAreaSqFt']).copy()
asmt_building_areas['BAT'] = (asmt_building_areas.loc[asmt_building_areas['BuildingAreaStndCode'] == 'BAT']['BuildingAreaSqFt']).copy()
asmt_building_areas

C:\Users\NPOLIM~1\AppData\Local\Temp\93/ipykernel_140360/4139555569.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asmt_building_areas['BAL'] = (asmt_building_areas.loc[asmt_building_areas['BuildingAreaStndCode'] == 'BAL']['BuildingAreaSqFt']).copy()
C:\Users\NPOLIM~1\AppData\Local\Temp\93/ipykernel_140360/4139555569.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asmt_building_areas['BAT'] = (asmt_building_areas.loc[asmt_building_areas['BuildingAreaStndCode'] == 'BAT']['BuildingAreaSqFt']).copy()


,RowID,BuildingAreaStndCode,BuildingAreaSqFt,BAL,BAT
4,0EA32702-0000-0000-32C9-000000000000,BAT,1068,<NA>,1068
84,83A32702-0000-0000-32C9-000000000000,BAT,9360,<NA>,9360
86,86A32702-0000-0000-32C9-000000000000,BAT,1600,<NA>,1600
87,87A32702-0000-0000-32C9-000000000000,BAT,2220,<NA>,2220
91,8BA32702-0000-0000-32C9-000000000000,BAT,3600,<NA>,3600
...,...,...,...,...,...
8463492,BE53E10B-0000-0000-33FF-000000000000,BAL,1978,1978,<NA>
8463495,BF53E10B-0000-0000-33FF-000000000000,BAL,2794,2794,<NA>
8463498,C153E10B-0000-0000-33FF-000000000000,BAL,3748,3748,<NA>
8463501,C553E10B-0000-0000-33FF-000000000000,BAL,1584,1584,<NA>


Count the number of duplicates in the modified TRANS BuildingAreas.

In [112]:
print("Number of entire row duplications: ", asmt_building_areas.duplicated().sum())
print("Number of rows with duplicate RowIDs: ", asmt_building_areas.duplicated(subset='RowID').sum())
print("Percent with duplicate RowIDs: ", asmt_building_areas.duplicated(subset='RowID').sum() / len(trans_building_areas.index) * 100)

Number of entire row duplications:  0
Number of rows with duplicate RowIDs:  86760
Percent with duplicate RowIDs:  5.389255033015088


Further, let's verify the count of 'BAL' and 'BAT' values in our output to the original data (they should be the same).

In [113]:
print("Count of 'BAL' in original data: ", len(store_original_df.loc[store_original_df['BuildingAreaStndCode'] == 'BAL'].index))
print("Count of 'BAL' in output data: ", len(asmt_building_areas.loc[trans_building_areas['BuildingAreaStndCode'] == 'BAL'].index))
print("Count of 'BAT' in original data: ", len(store_original_df.loc[store_original_df['BuildingAreaStndCode'] == 'BAT'].index))
print("Count of 'BAT' in output data: ", len(asmt_building_areas.loc[trans_building_areas['BuildingAreaStndCode'] == 'BAT'].index))

Count of 'BAL' in original data:  1374687
Count of 'BAL' in output data:  1374687
Count of 'BAT' in original data:  235183
Count of 'BAT' in output data:  235183


In [115]:
combined_temp = combined.merge(asmt_building_areas, how='left', on='RowID')

In [116]:
print("Count of entire row duplications: ", combined_temp.duplicated().sum())
print("Count of ImportParcelID duplications (some expected): ", combined_temp['ImportParcelID'].duplicated().sum())
print("Count of rows with duplicate RowID (some expected): ", combined_temp["RowID"].duplicated().sum())
print("Count of rows with duplicate TransId: ", combined_temp["TransId"].duplicated().sum())

Count of entire row duplications:  0
Count of ImportParcelID duplications (some expected):  3963785
Count of rows with duplicate RowID (some expected):  4231483
Count of rows with duplicate TransId:  286


Original count of duplicate RowID: 4231085
Increase: 4231501 - 4231085 = 416.

DETERMINATION: 5.4% of rows have duplicate RowIDs. This means that they had multiple values recorded for either 'BAL', 'BAT', or both.  
  
We are going to make the assumption that the last recorded value is correct, since the data might have been updated with more recent calculations.  
  
Therefore, drop rows with duplicate RowIDs, but keep the last observation with that RowID.

In [128]:
asmt_building_areas.drop_duplicates(subset='RowID', keep=False, inplace=True)

c:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [130]:
asmt_building_areas.tail(20)

,RowID,BAL,BAT
8463140,860E7E0B-0000-0000-3381-000000000000,<NA>,3200
8463173,BA0E7E0B-0000-0000-3381-000000000000,<NA>,5500
8463215,8FCDEF0B-0000-0000-3381-000000000000,<NA>,3380
8463265,7ACEEF0B-0000-0000-3381-000000000000,<NA>,4000
8463266,7BCEEF0B-0000-0000-3381-000000000000,<NA>,2314
8463272,81CEEF0B-0000-0000-3381-000000000000,<NA>,1224
8463276,85CEEF0B-0000-0000-3381-000000000000,<NA>,114460
8463370,48CFEF0B-0000-0000-3381-000000000000,<NA>,6000
8463434,FFCFEF0B-0000-0000-3381-000000000000,<NA>,114460
8463459,EB8DEB0B-0000-0000-32FB-000000000000,<NA>,596


Left join combined and TRANS BuildingAreas. 

In [131]:
combined = combined.merge(trans_building_areas, how='left', on='RowID')
combined

,TransId,PropertyHouseNumber_x,PropertyFullStreetAddress_x,PropertyCity_x,PropertyZip_x,PropertyZip4_x,PropertySequenceNumber,FIPS_x,ImportParcelID,RowID,...,PropertyUseStndCode,LoanTypeStndCode,TotalDelinquentAmount,DelinquentAsOfDate,BatchID_y,FCDE,BuildingAreaStndCode,BuildingAreaSqFt,BAL,BAT
0,148470374,286,286 Fiddlers Trl,Auburn,30011,3350,1,13013,36214022,06952802-0000-0000-32D5-000000000000,...,SR,NaN,<NA>,<NA>,1002049440,False,NaN,<NA>,<NA>,<NA>
1,148470375,1542,1542 Oakleaf Dr,Auburn,30011,3268,1,13013,36213235,F3912802-0000-0000-32D5-000000000000,...,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>
2,148470605,<NA>,WILLIAMS RD,AUBURN,30203,,1,13013,<NA>,<NA>,...,SR,NaN,<NA>,<NA>,1002049440,False,NaN,<NA>,<NA>,<NA>
3,148470703,505,505 Yargo Ln,Winder,30680,3954,1,13013,36234911,9FE62802-0000-0000-32D5-000000000000,...,SR,NaN,<NA>,<NA>,1002049440,False,NaN,<NA>,<NA>,<NA>
4,148471097,384,384 Scenic Ln,Auburn,30011,2516,1,13013,36213478,E6922802-0000-0000-32D5-000000000000,...,UL,NaN,<NA>,<NA>,1002049440,False,NaN,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5296826,578457745,5787,5787 Mason Jones Dr,Powder Springs,30127,4761,1,13067,37184519,07643702-0000-0000-330B-000000000000,...,NaN,NaN,<NA>,<NA>,<NA>,NaN,BAL,2572,2572,<NA>
5296827,578457746,724,724 Registry Run NW,Kennesaw,30152,2870,1,13067,37229517,CD133802-0000-0000-330B-000000000000,...,NaN,NaN,<NA>,<NA>,<NA>,NaN,BAL,2973,2973,<NA>
5296828,578457747,4834,4834 Olde Mill Dr,Marietta,30066,1159,1,13067,37000623,AF953402-0000-0000-330B-000000000000,...,NaN,NaN,<NA>,<NA>,<NA>,NaN,BAL,1474,1474,<NA>
5296829,578457748,5138,5138 Lakeview Ct,Austell,30106,2737,1,13067,37181434,<NA>,...,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>


Examine duplication issues. The number of duplicate RowIDs should not have increased from previously (previous number: 3,964,251). There should also be no duplications of TransId.

In [132]:
print("Count of entire row duplications: ", combined.duplicated().sum())
print("Count of ImportParcelID duplications (some expected): ", combined['ImportParcelID'].duplicated().sum())
print("Count of rows with duplicate RowID (some expected): ", combined["RowID"].duplicated().sum())
print("Count of rows with duplicate TransId: ", combined["TransId"].duplicated().sum())

Count of entire row duplications:  0
Count of ImportParcelID duplications (some expected):  3963785
Count of rows with duplicate RowID (some expected):  4231483
Count of rows with duplicate TransId:  286


Examine duplications.

In [133]:
print("Entire row duplicated (auto dropped): ", asmt_building.duplicated().sum())
asmt_building.drop_duplicates(inplace=True)

print("--------")
num_dup = asmt_building['RowID'].duplicated().sum()
print("Duplicates (RowID): ", num_dup)
print("Percent: ", num_dup / len(asmt_building.index) * 100)

Entire row duplicated (auto dropped):  0
--------
Duplicates (RowID):  120475
Percent:  2.4516851659768983


RowID duplications are likely caused by multiple buildings on the same parcel and are an insignificant portion of data.  
  
Therefore, drop duplicate RowIDs, including the first occurance of that RowID.

In [134]:
init_size = len(asmt_building.index)
asmt_building.drop_duplicates(subset='RowID', keep=False, inplace=True)

print("Percent dropped: ", (init_size - len(asmt_building.index)) / init_size * 100)

Percent dropped:  4.303814006076964


The percent dropped is higher than the percentage of duplicates calculated previously because we are also dropping the first occurance. We do this because we do not want to investigate parcels with more than one building on them.
  
Further, this percent would likely be lower after account for properties in the MSA and for SFH. But we want to drop these before we merge to avoid issues, so we don't have that exact percent.  

We could potentially change this later but it does not seem important.

In [135]:
combined = combined.merge(asmt_building, how='left', on='RowID')

In [136]:
combined

,TransId,PropertyHouseNumber_x,PropertyFullStreetAddress_x,PropertyCity_x,PropertyZip_x,PropertyZip4_x,PropertySequenceNumber,FIPS_x,ImportParcelID,RowID,...,BAT,NoOfUnits_y,PropertyCountyLandUseCode_y,PropertyLandUseStndCode_y,YearBuilt_y,NoOfStories_y,TotalRooms_y,TotalBedrooms_y,TotalCalculatedBathCount_y,StoryTypeStndCode_y
0,148470374,286,286 Fiddlers Trl,Auburn,30011,3350,1,13013,36214022,06952802-0000-0000-32D5-000000000000,...,<NA>,<NA>,1,RR101,1988,7.0,5,3,3.0,SPL
1,148470375,1542,1542 Oakleaf Dr,Auburn,30011,3268,1,13013,36213235,F3912802-0000-0000-32D5-000000000000,...,<NA>,1,1,RR101,1990,1.0,5,3,2.0,SGL
2,148470605,<NA>,WILLIAMS RD,AUBURN,30203,,1,13013,<NA>,<NA>,...,<NA>,<NA>,NaN,NaN,<NA>,NaN,<NA>,NaN,NaN,NaN
3,148470703,505,505 Yargo Ln,Winder,30680,3954,1,13013,36234911,9FE62802-0000-0000-32D5-000000000000,...,<NA>,2400,1,RR101,1970,1.0,6,3,2.0,SGL
4,148471097,384,384 Scenic Ln,Auburn,30011,2516,1,13013,36213478,E6922802-0000-0000-32D5-000000000000,...,<NA>,<NA>,1,RR101,2006,11.0,<NA>,NaN,3.0,SPF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5296826,578457745,5787,5787 Mason Jones Dr,Powder Springs,30127,4761,1,13067,37184519,07643702-0000-0000-330B-000000000000,...,<NA>,<NA>,101,RR101,1999,2.0,7,4,2.5,NaN
5296827,578457746,724,724 Registry Run NW,Kennesaw,30152,2870,1,13067,37229517,CD133802-0000-0000-330B-000000000000,...,<NA>,<NA>,101,RR101,2000,2.0,9,5,4.0,NaN
5296828,578457747,4834,4834 Olde Mill Dr,Marietta,30066,1159,1,13067,37000623,AF953402-0000-0000-330B-000000000000,...,<NA>,<NA>,101,RR101,1986,2.0,6,3,2.5,NaN
5296829,578457748,5138,5138 Lakeview Ct,Austell,30106,2737,1,13067,37181434,<NA>,...,<NA>,<NA>,NaN,NaN,<NA>,NaN,<NA>,NaN,NaN,NaN


Examine duplicates in combined after merging ASMT Building.

In [137]:
print("Count of entire row duplications: ", combined.duplicated().sum())
print("Count of ImportParcelID duplications (some expected): ", combined['ImportParcelID'].duplicated().sum())
print("Count of rows with duplicate RowID (some expected): ", combined["RowID"].duplicated().sum())
print("Count of rows with duplicate TransId: ", combined["TransId"].duplicated().sum())

Count of entire row duplications:  0
Count of ImportParcelID duplications (some expected):  3963785
Count of rows with duplicate RowID (some expected):  4231483
Count of rows with duplicate TransId:  286


### Step 6  
Left join combined and ASMT Value.

In [138]:
asmt_value = pd.read_csv(
        '../20221103_GA/ZAsmt/Value.txt',
        sep='|',
        on_bad_lines='skip',
        encoding='latin-1',
        quoting=csv.QUOTE_NONE,
        header=None,
        usecols=[0, 3, 4],
        names=["RowID", "TotalAssessedValue", "AssessmentYear"],
        dtype={"RowID": pd.StringDtype(), "TotalAssessedValue": "Int32", "AssessmentYear": "Int32"})

In [139]:
asmt_value

,RowID,TotalAssessedValue,AssessmentYear
0,05A32702-0000-0000-32C9-000000000000,12017,2021
1,06A32702-0000-0000-32C9-000000000000,707,2021
2,07A32702-0000-0000-32C9-000000000000,23300,2021
3,08A32702-0000-0000-32C9-000000000000,1432,2021
4,09A32702-0000-0000-32C9-000000000000,35623,2021
...,...,...,...
4793471,A473120C-0000-0000-3405-000000000000,486,2021
4793472,A573120C-0000-0000-3405-000000000000,58210,2021
4793473,A673120C-0000-0000-3405-000000000000,7517,2021
4793474,A773120C-0000-0000-3405-000000000000,88935,2021


Examine duplicates in ASMT Value.

In [140]:
print("Entire row duplicated (auto dropped): ", asmt_value.duplicated().sum())
asmt_value.drop_duplicates(inplace=True)

print("--------")
num_dup = asmt_value['RowID'].duplicated().sum()
print("Duplicates (RowID): ", num_dup)
print("Percent: ", num_dup / len(asmt_value.index) * 100)

Entire row duplicated (auto dropped):  0
--------
Duplicates (RowID):  0
Percent:  0.0


In [141]:
combined = combined.merge(asmt_value, how='left', on='RowID')

In [142]:
combined

,TransId,PropertyHouseNumber_x,PropertyFullStreetAddress_x,PropertyCity_x,PropertyZip_x,PropertyZip4_x,PropertySequenceNumber,FIPS_x,ImportParcelID,RowID,...,PropertyCountyLandUseCode_y,PropertyLandUseStndCode_y,YearBuilt_y,NoOfStories_y,TotalRooms_y,TotalBedrooms_y,TotalCalculatedBathCount_y,StoryTypeStndCode_y,TotalAssessedValue,AssessmentYear
0,148470374,286,286 Fiddlers Trl,Auburn,30011,3350,1,13013,36214022,06952802-0000-0000-32D5-000000000000,...,1,RR101,1988,7.0,5,3,3.0,SPL,62058,2021
1,148470375,1542,1542 Oakleaf Dr,Auburn,30011,3268,1,13013,36213235,F3912802-0000-0000-32D5-000000000000,...,1,RR101,1990,1.0,5,3,2.0,SGL,54358,2021
2,148470605,<NA>,WILLIAMS RD,AUBURN,30203,,1,13013,<NA>,<NA>,...,NaN,NaN,<NA>,NaN,<NA>,NaN,NaN,NaN,<NA>,<NA>
3,148470703,505,505 Yargo Ln,Winder,30680,3954,1,13013,36234911,9FE62802-0000-0000-32D5-000000000000,...,1,RR101,1970,1.0,6,3,2.0,SGL,58143,2021
4,148471097,384,384 Scenic Ln,Auburn,30011,2516,1,13013,36213478,E6922802-0000-0000-32D5-000000000000,...,1,RR101,2006,11.0,<NA>,NaN,3.0,SPF,76869,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5296826,578457745,5787,5787 Mason Jones Dr,Powder Springs,30127,4761,1,13067,37184519,07643702-0000-0000-330B-000000000000,...,101,RR101,1999,2.0,7,4,2.5,NaN,86896,2021
5296827,578457746,724,724 Registry Run NW,Kennesaw,30152,2870,1,13067,37229517,CD133802-0000-0000-330B-000000000000,...,101,RR101,2000,2.0,9,5,4.0,NaN,153648,2021
5296828,578457747,4834,4834 Olde Mill Dr,Marietta,30066,1159,1,13067,37000623,AF953402-0000-0000-330B-000000000000,...,101,RR101,1986,2.0,6,3,2.5,NaN,88576,2021
5296829,578457748,5138,5138 Lakeview Ct,Austell,30106,2737,1,13067,37181434,<NA>,...,NaN,NaN,<NA>,NaN,<NA>,NaN,NaN,NaN,<NA>,<NA>


Examine duplicates in combined after merging ASMT Value.

In [143]:
print("Count of entire row duplications: ", combined.duplicated().sum())
print("Count of ImportParcelID duplications (some expected): ", combined['ImportParcelID'].duplicated().sum())
print("Count of rows with duplicate RowID (some expected): ", combined["RowID"].duplicated().sum())
print("Count of rows with duplicate TransId: ", combined["TransId"].duplicated().sum())

Count of entire row duplications:  0
Count of ImportParcelID duplications (some expected):  3963785
Count of rows with duplicate RowID (some expected):  4231483
Count of rows with duplicate TransId:  286


### Step 7
Drop rows with 'TotalAssessedValue' == NA or 0, AssessmentYear == NA

In [144]:
init_size = len(combined.index)

combined = combined.loc[combined['TotalAssessedValue'].notna()]
new_size = len(combined.index)
print("Number dropped where TotalAssessedValue == NA: ", init_size - new_size)

combined = combined.loc[combined['TotalAssessedValue'] != 0]
prev_size = new_size
new_size = len(combined.index)
print("Number dropped where TotalAssessedValue == 0: ", init_size - new_size)

combined = combined.loc[combined['AssessmentYear'].notna()]
prev_size = new_size
new_size = len(combined.index)
print("Number dropped where AssessmentYear == NA: ", init_size - new_size)
print("--------")
print("Percent dropped (total): ", (init_size - new_size) / init_size * 100)

Number dropped where TotalAssessedValue == NA:  1124286
Number dropped where TotalAssessedValue == 0:  1124485
Number dropped where AssessmentYear == NA:  1124485
--------
Percent dropped (total):  21.229391687218264


In [145]:
combined_source = combined.copy()

### Step 8  
Merge TRANS BuyerName to BuyerMailAddress, merge TRANS SellerName to SellerMailAddress, for the first two observations of each transcation, then merge these files onto the main dataframe.

Read in BuyerName and print a sample.

In [146]:
BuyerName = pd.read_csv(
    '../20221103_GA/ZTrans/BuyerName.txt',
    sep='|',
    on_bad_lines='skip',
    quoting=csv.QUOTE_NONE,
    header=None,
    usecols=[0, 3, 4, 5],
    names=["TransId", "BuyerIndividualFullName", "BuyerNonIndividualName", "BuyerNameSequenceNumber"],
    dtype={"TransId": "Int32", "BuyerFirstMiddleName": pd.StringDtype(), "BuyerLastName": pd.StringDtype(), "BuyerIndividualFullName": pd.StringDtype(), "BuyerNonIndividualName": pd.StringDtype(),
           "BuyerNameSequenceNumber": "Int16"})

In [147]:
BuyerName

,TransId,BuyerIndividualFullName,BuyerNonIndividualName,BuyerNameSequenceNumber
0,148390440,CURT MATULICH,<NA>,1
1,148390440,REBECCA MATULICH,<NA>,2
2,148390441,LIZZIE M SAILEM,<NA>,1
3,148390442,ROBERT RICHBURG,<NA>,1
4,148390442,TAMMY RICHBURG,<NA>,2
...,...,...,...,...
6926114,578458860,KEVIN MUNDLE,<NA>,2
6926115,578458861,SAMANTHA ANTONIA THORPE,<NA>,1
6926116,578458862,<NA>,"BCB ENTITLES, LLC",1
6926117,578458863,<NA>,"MADISON STAR, LLC",1


Count the number of rows with duplicate TransIds. 

In [148]:
num_dup = BuyerName.duplicated(subset='TransId').sum()
print("Count of rows with duplicate TransId: ", num_dup)
print("Percent: ", num_dup / len(BuyerName.index) * 100)

Count of rows with duplicate TransId:  1532706
Percent:  22.129362778779864


Read in BuyerMailAddress.

In [149]:
BuyerMailAddress = pd.read_csv(
    '../20221103_GA/ZTrans/BuyerMailAddress.txt',
    sep='|',
    on_bad_lines='skip',
    encoding='latin-1',
    quoting=csv.QUOTE_NONE,
    header=None,
    usecols=[0, 1, 11, 12, 14, 15],
    names=["TransId", "BuyerMailSequenceNumber", "BuyerMailFullStreetAddress", "BuyerMailCity", "BuyerMailZip", "BuyerMailZip4"],
    dtype={"TransId": "Int32", "BuyerMailSequenceNumber": "Int32", "BuyerMailFullStreetAddress": pd.StringDtype(),
           "BuyerMailCity": "category", "BuyerMailZip": "category", "BuyerMailZip4": "category"})

In [150]:
BuyerMailAddress

,TransId,BuyerMailSequenceNumber,BuyerMailFullStreetAddress,BuyerMailCity,BuyerMailZip,BuyerMailZip4
0,148390440,1,4540 Sundance Cir,Cumming,30028,3417
1,148390441,1,180 Idell St,Baxley,31513,7817
2,148390442,1,473 James Kent Rd,Baxley,31513,7404
3,148390443,1,38 Cleve White Rd,Baxley,31513,2270
4,148390445,1,E MARTIN 900 ST,FOLKSTON,31537,
...,...,...,...,...,...,...
5521681,578367886,1,301 Commerce St,Fort Worth,76102,4140
5521682,578367887,1,301 Commerce St,Fort Worth,76102,4140
5521683,578367888,1,141 Fox Rd,Macon,31217,2229
5521684,578367888,2,<NA>,NaN,NaN,NaN


Count the number of rows with duplicate TransIds. 

In [151]:
num_dup = BuyerMailAddress.duplicated(subset='TransId').sum()
print("Count of rows with duplicate TransId: ", num_dup)
print("Percent: ", num_dup / len(BuyerMailAddress.index) * 100)

Count of rows with duplicate TransId:  152174
Percent:  2.7559336043375158


There are a lot fewer rows with duplicate TransIds in BuyerName than in BuyerMailAddress. This means that many BuyerNames do not have associated BuyerMailAddresses. As a result, in the merged dataframe, there will be a lot of rows with null values which could not be filled in by BuyerName.

Merge BuyerName and BuyerMailAddress using a left merge on ['TransId', 'BuyerNameSequenceNumber' = 'BuyerMailSequenceNumber']

In [152]:
merged = BuyerName.merge(BuyerMailAddress, how='left', left_on=['TransId', 'BuyerNameSequenceNumber'], right_on=['TransId', 'BuyerMailSequenceNumber'])

In [153]:
merged

,TransId,BuyerIndividualFullName,BuyerNonIndividualName,BuyerNameSequenceNumber,BuyerMailSequenceNumber,BuyerMailFullStreetAddress,BuyerMailCity,BuyerMailZip,BuyerMailZip4
0,148390440,CURT MATULICH,<NA>,1,1,4540 Sundance Cir,Cumming,30028,3417
1,148390440,REBECCA MATULICH,<NA>,2,<NA>,<NA>,NaN,NaN,NaN
2,148390441,LIZZIE M SAILEM,<NA>,1,1,180 Idell St,Baxley,31513,7817
3,148390442,ROBERT RICHBURG,<NA>,1,1,473 James Kent Rd,Baxley,31513,7404
4,148390442,TAMMY RICHBURG,<NA>,2,<NA>,<NA>,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6926114,578458860,KEVIN MUNDLE,<NA>,2,<NA>,<NA>,NaN,NaN,NaN
6926115,578458861,SAMANTHA ANTONIA THORPE,<NA>,1,<NA>,<NA>,NaN,NaN,NaN
6926116,578458862,<NA>,"BCB ENTITLES, LLC",1,<NA>,<NA>,NaN,NaN,NaN
6926117,578458863,<NA>,"MADISON STAR, LLC",1,<NA>,<NA>,NaN,NaN,NaN


In [154]:
merged[merged['BuyerMailSequenceNumber'] == 2 & merged['BuyerNonIndividualName'].notna()]

,TransId,BuyerIndividualFullName,BuyerNonIndividualName,BuyerNameSequenceNumber,BuyerMailSequenceNumber,BuyerMailFullStreetAddress,BuyerMailCity,BuyerMailZip,BuyerMailZip4


There are no cases where two addresses are recorded when the buyer is a NonIndividual

Examine the number of rows without BuyerMailAddress information.

In [155]:
print("Number of rows without BuyerMailAddress info: ", len(merged[merged['BuyerMailSequenceNumber'].isna()].index))
print("Percent: ", len(merged[merged['BuyerMailSequenceNumber'].isna()].index) / len(merged.index) * 100)

Number of rows without BuyerMailAddress info:  1410943
Percent:  20.37133638622149


Examine the distribution of BuyerNameSequenceNumber, which indexes each buyer involved in a transcation. Consider the percent of transcations involving multiple buyers.

In [156]:
merged['BuyerNameSequenceNumber'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

1     77.9%
2     21.7%
3      0.2%
4      0.1%
5      0.0%
6      0.0%
7      0.0%
8      0.0%
9      0.0%
10     0.0%
11     0.0%
12     0.0%
13     0.0%
14     0.0%
15     0.0%
16     0.0%
17     0.0%
18     0.0%
19     0.0%
20     0.0%
21     0.0%
22     0.0%
23     0.0%
24     0.0%
25     0.0%
26     0.0%
27     0.0%
28     0.0%
29     0.0%
30     0.0%
31     0.0%
Name: BuyerNameSequenceNumber, dtype: object

DETERMINATION: Since transcations involving 2 buyers captures (77.9 + 21.7 - .3) = 99.3% of observations, simply drop any transcations with more than 2 buyers.  
  
However, we cannot simply drop observations where BuyerNameSequenceNumber > 2; in that case, we would still retain these many-buyer transcations by keeping their first 2 Buyers.  
  
Since this is more complicated, let's just only take the first 2 buyers from all transcations, ignoring how many buyers total were involved. Since transcations with more than two buyers is so significant, this method should be sufficient. Further, this percentage might get even less significant when accounting for SFH and the Atlanta MSA. Let's test this by first merging the data via a left-join to the main dataframe, then looking at the distribution again.


In [157]:
merged.columns

Index(['TransId', 'BuyerIndividualFullName', 'BuyerNonIndividualName',
       'BuyerNameSequenceNumber', 'BuyerMailSequenceNumber',
       'BuyerMailFullStreetAddress', 'BuyerMailCity', 'BuyerMailZip',
       'BuyerMailZip4'],
      dtype='object')

In [158]:
merged['BuyerIndividualFullName'] = merged['BuyerIndividualFullName'].fillna("NA")
merged['BuyerNonIndividualName'] = merged['BuyerNonIndividualName'].fillna("NA")

In [159]:
merged

,TransId,BuyerIndividualFullName,BuyerNonIndividualName,BuyerNameSequenceNumber,BuyerMailSequenceNumber,BuyerMailFullStreetAddress,BuyerMailCity,BuyerMailZip,BuyerMailZip4
0,148390440,CURT MATULICH,NA,1,1,4540 Sundance Cir,Cumming,30028,3417
1,148390440,REBECCA MATULICH,NA,2,<NA>,<NA>,NaN,NaN,NaN
2,148390441,LIZZIE M SAILEM,NA,1,1,180 Idell St,Baxley,31513,7817
3,148390442,ROBERT RICHBURG,NA,1,1,473 James Kent Rd,Baxley,31513,7404
4,148390442,TAMMY RICHBURG,NA,2,<NA>,<NA>,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6926114,578458860,KEVIN MUNDLE,NA,2,<NA>,<NA>,NaN,NaN,NaN
6926115,578458861,SAMANTHA ANTONIA THORPE,NA,1,<NA>,<NA>,NaN,NaN,NaN
6926116,578458862,NA,"BCB ENTITLES, LLC",1,<NA>,<NA>,NaN,NaN,NaN
6926117,578458863,NA,"MADISON STAR, LLC",1,<NA>,<NA>,NaN,NaN,NaN


Strip commas from BuyerNonIndividualName.

In [160]:
merged['BuyerNonIndividualName'] = merged['BuyerNonIndividualName'].str.replace(',', '')

In [161]:
BuyerNameAgg = merged.groupby("TransId", as_index=False)['BuyerIndividualFullName', 'BuyerNonIndividualName'].agg(','.join)
BuyerNameAgg

C:\Users\NPOLIM~1\AppData\Local\Temp\93/ipykernel_140360/1590539918.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  BuyerNameAgg = merged.groupby("TransId", as_index=False)['BuyerIndividualFullName', 'BuyerNonIndividualName'].agg(','.join)


,TransId,BuyerIndividualFullName,BuyerNonIndividualName
0,148390440,"CURT MATULICH,REBECCA MATULICH","NA,NA"
1,148390441,LIZZIE M SAILEM,NA
2,148390442,"ROBERT RICHBURG,TAMMY RICHBURG","NA,NA"
3,148390443,SHERMAN WILLIAMS,NA
4,148390445,JOHN A BOATRIGHT,NA
...,...,...,...
5393408,578458860,"ASHLEIGH MUNDLE,KEVIN MUNDLE","NA,NA"
5393409,578458861,SAMANTHA ANTONIA THORPE,NA
5393410,578458862,NA,BCB ENTITLES LLC
5393411,578458863,NA,MADISON STAR LLC


In [162]:
merged[['BuyerIndividualFullName', 'BuyerNonIndividualName', 'BuyerNameSequenceNumber']]

,BuyerIndividualFullName,BuyerNonIndividualName,BuyerNameSequenceNumber
0,CURT MATULICH,NA,1
1,REBECCA MATULICH,NA,2
2,LIZZIE M SAILEM,NA,1
3,ROBERT RICHBURG,NA,1
4,TAMMY RICHBURG,NA,2
...,...,...,...
6926114,KEVIN MUNDLE,NA,2
6926115,SAMANTHA ANTONIA THORPE,NA,1
6926116,NA,BCB ENTITLES LLC,1
6926117,NA,MADISON STAR LLC,1


In [163]:
BuyerNameAgg['BuyerIndividualFullName1'] = BuyerNameAgg[BuyerNameAgg['BuyerIndividualFullName'].str.contains(',')]['BuyerIndividualFullName'].str.split(',').apply(lambda x: x[1])
BuyerNameAgg

,TransId,BuyerIndividualFullName,BuyerNonIndividualName,BuyerIndividualFullName1
0,148390440,"CURT MATULICH,REBECCA MATULICH","NA,NA",REBECCA MATULICH
1,148390441,LIZZIE M SAILEM,NA,NaN
2,148390442,"ROBERT RICHBURG,TAMMY RICHBURG","NA,NA",TAMMY RICHBURG
3,148390443,SHERMAN WILLIAMS,NA,NaN
4,148390445,JOHN A BOATRIGHT,NA,NaN
...,...,...,...,...
5393408,578458860,"ASHLEIGH MUNDLE,KEVIN MUNDLE","NA,NA",KEVIN MUNDLE
5393409,578458861,SAMANTHA ANTONIA THORPE,NA,NaN
5393410,578458862,NA,BCB ENTITLES LLC,NaN
5393411,578458863,NA,MADISON STAR LLC,NaN


In [164]:
BuyerNameAgg['BuyerIndividualFullName'] = BuyerNameAgg[BuyerNameAgg['BuyerIndividualFullName'].notna()]['BuyerIndividualFullName'].str.split(',').apply(lambda x: x[0])
BuyerNameAgg

,TransId,BuyerIndividualFullName,BuyerNonIndividualName,BuyerIndividualFullName1
0,148390440,CURT MATULICH,"NA,NA",REBECCA MATULICH
1,148390441,LIZZIE M SAILEM,NA,NaN
2,148390442,ROBERT RICHBURG,"NA,NA",TAMMY RICHBURG
3,148390443,SHERMAN WILLIAMS,NA,NaN
4,148390445,JOHN A BOATRIGHT,NA,NaN
...,...,...,...,...
5393408,578458860,ASHLEIGH MUNDLE,"NA,NA",KEVIN MUNDLE
5393409,578458861,SAMANTHA ANTONIA THORPE,NA,NaN
5393410,578458862,NA,BCB ENTITLES LLC,NaN
5393411,578458863,NA,MADISON STAR LLC,NaN


Do the same for BuyerNonIndividualName

In [165]:
BuyerNameAgg['BuyerNonIndividualName1'] = BuyerNameAgg[BuyerNameAgg['BuyerNonIndividualName'].str.contains(',')]['BuyerNonIndividualName'].str.split(',').apply(lambda x: x[1])
BuyerNameAgg

,TransId,BuyerIndividualFullName,BuyerNonIndividualName,BuyerIndividualFullName1,BuyerNonIndividualName1
0,148390440,CURT MATULICH,"NA,NA",REBECCA MATULICH,NA
1,148390441,LIZZIE M SAILEM,NA,NaN,NaN
2,148390442,ROBERT RICHBURG,"NA,NA",TAMMY RICHBURG,NA
3,148390443,SHERMAN WILLIAMS,NA,NaN,NaN
4,148390445,JOHN A BOATRIGHT,NA,NaN,NaN
...,...,...,...,...,...
5393408,578458860,ASHLEIGH MUNDLE,"NA,NA",KEVIN MUNDLE,NA
5393409,578458861,SAMANTHA ANTONIA THORPE,NA,NaN,NaN
5393410,578458862,NA,BCB ENTITLES LLC,NaN,NaN
5393411,578458863,NA,MADISON STAR LLC,NaN,NaN


In [166]:
BuyerNameAgg['BuyerNonIndividualName'] = BuyerNameAgg[BuyerNameAgg['BuyerNonIndividualName'].notna()]['BuyerNonIndividualName'].str.split(',').apply(lambda x: x[0])
BuyerNameAgg

,TransId,BuyerIndividualFullName,BuyerNonIndividualName,BuyerIndividualFullName1,BuyerNonIndividualName1
0,148390440,CURT MATULICH,NA,REBECCA MATULICH,NA
1,148390441,LIZZIE M SAILEM,NA,NaN,NaN
2,148390442,ROBERT RICHBURG,NA,TAMMY RICHBURG,NA
3,148390443,SHERMAN WILLIAMS,NA,NaN,NaN
4,148390445,JOHN A BOATRIGHT,NA,NaN,NaN
...,...,...,...,...,...
5393408,578458860,ASHLEIGH MUNDLE,NA,KEVIN MUNDLE,NA
5393409,578458861,SAMANTHA ANTONIA THORPE,NA,NaN,NaN
5393410,578458862,NA,BCB ENTITLES LLC,NaN,NaN
5393411,578458863,NA,MADISON STAR LLC,NaN,NaN


In [167]:
BuyerNameAgg = BuyerNameAgg.fillna("NA")

No transcations where there are both Individual buyers and NonIndividual buyers.

In [168]:
BuyerNameAgg[BuyerNameAgg['BuyerIndividualFullName'] != "NA"][BuyerNameAgg['BuyerNonIndividualName'] != "NA"]

C:\Users\NPOLIM~1\AppData\Local\Temp\93/ipykernel_140360/1397266248.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  BuyerNameAgg[BuyerNameAgg['BuyerIndividualFullName'] != "NA"][BuyerNameAgg['BuyerNonIndividualName'] != "NA"]


,TransId,BuyerIndividualFullName,BuyerNonIndividualName,BuyerIndividualFullName1,BuyerNonIndividualName1


In [169]:
BuyerAddressAgg = merged[['TransId', 'BuyerMailFullStreetAddress', 'BuyerMailCity', 'BuyerMailZip', 'BuyerMailZip4']]
BuyerAddressAgg.drop_duplicates(subset='TransId', keep='first', inplace=True)

c:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [170]:
BuyerNameAgg.duplicated(subset='TransId').sum()

0

In [171]:
BuyerAddressAgg.duplicated(subset='TransId').sum()

0

In [172]:
buyerMerged = BuyerNameAgg.merge(BuyerAddressAgg, on='TransId', how='left')

In [173]:
buyerMerged

,TransId,BuyerIndividualFullName,BuyerNonIndividualName,BuyerIndividualFullName1,BuyerNonIndividualName1,BuyerMailFullStreetAddress,BuyerMailCity,BuyerMailZip,BuyerMailZip4
0,148390440,CURT MATULICH,NA,REBECCA MATULICH,NA,4540 Sundance Cir,Cumming,30028,3417
1,148390441,LIZZIE M SAILEM,NA,NA,NA,180 Idell St,Baxley,31513,7817
2,148390442,ROBERT RICHBURG,NA,TAMMY RICHBURG,NA,473 James Kent Rd,Baxley,31513,7404
3,148390443,SHERMAN WILLIAMS,NA,NA,NA,38 Cleve White Rd,Baxley,31513,2270
4,148390445,JOHN A BOATRIGHT,NA,NA,NA,E MARTIN 900 ST,FOLKSTON,31537,
...,...,...,...,...,...,...,...,...,...
5393408,578458860,ASHLEIGH MUNDLE,NA,KEVIN MUNDLE,NA,<NA>,NaN,NaN,NaN
5393409,578458861,SAMANTHA ANTONIA THORPE,NA,NA,NA,<NA>,NaN,NaN,NaN
5393410,578458862,NA,BCB ENTITLES LLC,NA,NA,<NA>,NaN,NaN,NaN
5393411,578458863,NA,MADISON STAR LLC,NA,NA,<NA>,NaN,NaN,NaN


In [174]:
BuyerAddressAgg

,TransId,BuyerMailFullStreetAddress,BuyerMailCity,BuyerMailZip,BuyerMailZip4
0,148390440,4540 Sundance Cir,Cumming,30028,3417
2,148390441,180 Idell St,Baxley,31513,7817
3,148390442,473 James Kent Rd,Baxley,31513,7404
5,148390443,38 Cleve White Rd,Baxley,31513,2270
6,148390445,E MARTIN 900 ST,FOLKSTON,31537,
...,...,...,...,...,...
6926113,578458860,<NA>,NaN,NaN,NaN
6926115,578458861,<NA>,NaN,NaN,NaN
6926116,578458862,<NA>,NaN,NaN,NaN
6926117,578458863,<NA>,NaN,NaN,NaN


In [175]:
mail_temp = BuyerAddressAgg[BuyerAddressAgg.duplicated(subset='TransId')]
mail_temp

,TransId,BuyerMailFullStreetAddress,BuyerMailCity,BuyerMailZip,BuyerMailZip4


In [176]:
mail_temp.loc[mail_temp['BuyerMailFullStreetAddress'] != "NA"]

,TransId,BuyerMailFullStreetAddress,BuyerMailCity,BuyerMailZip,BuyerMailZip4


In [178]:
BuyerAddressAgg = BuyerAddressAgg.drop_duplicates(subset='TransId')

In [179]:
BuyerNameAgg

,TransId,BuyerIndividualFullName,BuyerNonIndividualName,BuyerIndividualFullName1,BuyerNonIndividualName1
0,148390440,CURT MATULICH,NA,REBECCA MATULICH,NA
1,148390441,LIZZIE M SAILEM,NA,NA,NA
2,148390442,ROBERT RICHBURG,NA,TAMMY RICHBURG,NA
3,148390443,SHERMAN WILLIAMS,NA,NA,NA
4,148390445,JOHN A BOATRIGHT,NA,NA,NA
...,...,...,...,...,...
5393408,578458860,ASHLEIGH MUNDLE,NA,KEVIN MUNDLE,NA
5393409,578458861,SAMANTHA ANTONIA THORPE,NA,NA,NA
5393410,578458862,NA,BCB ENTITLES LLC,NA,NA
5393411,578458863,NA,MADISON STAR LLC,NA,NA


SellerName

In [180]:
SellerName = pd.read_csv(
    '../20221103_GA/ZTrans/SellerName.txt',
    sep='|',
    on_bad_lines='skip',
    encoding='latin-1',
    quoting=csv.QUOTE_NONE,
    header=None,
    usecols=[0, 3, 4, 5],
    names=["TransId", "SellerIndividualFullName", "SellerNonIndividualName", "SellerNameSequenceNumber"],
    dtype={"TransId": "Int32", "SellerIndividualFullName": pd.StringDtype(), "SellerNonIndividualName": pd.StringDtype(), "SellerNameSequenceNumber": "Int32"})

In [181]:
SellerName

,TransId,SellerIndividualFullName,SellerNonIndividualName,SellerNameSequenceNumber
0,148390440,DAVID ALLEN JOHNSON,<NA>,1
1,148390441,LENNOX M MORRIS,<NA>,1
2,148390442,JAMES MITCHELL WILLIAMS,<NA>,1
3,148390443,HERMAN MOBLEY,<NA>,1
4,148390445,<NA>,UNITED STATES DEPARTMENT O,1
...,...,...,...,...
5614666,578376772,PAMELA D COCHRAN,<NA>,1
5614667,578376773,BEN H HELTON,<NA>,1
5614668,578376774,AMANY T RODRIGUEZ,<NA>,1
5614669,578376775,CHARLENE N HOLLOWAY,<NA>,1


In [182]:
SellerName[SellerName['SellerIndividualFullName'].isna()][SellerName['SellerNonIndividualName'].isna()]

C:\Users\NPOLIM~1\AppData\Local\Temp\93/ipykernel_140360/510830998.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  SellerName[SellerName['SellerIndividualFullName'].isna()][SellerName['SellerNonIndividualName'].isna()]


,TransId,SellerIndividualFullName,SellerNonIndividualName,SellerNameSequenceNumber


In [183]:
SellerName[SellerName[['SellerIndividualFullName','SellerNonIndividualName']].isna().any(1)]

,TransId,SellerIndividualFullName,SellerNonIndividualName,SellerNameSequenceNumber
0,148390440,DAVID ALLEN JOHNSON,<NA>,1
1,148390441,LENNOX M MORRIS,<NA>,1
2,148390442,JAMES MITCHELL WILLIAMS,<NA>,1
3,148390443,HERMAN MOBLEY,<NA>,1
4,148390445,<NA>,UNITED STATES DEPARTMENT O,1
...,...,...,...,...
5614666,578376772,PAMELA D COCHRAN,<NA>,1
5614667,578376773,BEN H HELTON,<NA>,1
5614668,578376774,AMANY T RODRIGUEZ,<NA>,1
5614669,578376775,CHARLENE N HOLLOWAY,<NA>,1


In [184]:
SellerName['SellerNameSequenceNumber'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

1     95.1%
2      4.2%
3      0.5%
4      0.1%
5      0.0%
6      0.0%
7      0.0%
8      0.0%
9      0.0%
10     0.0%
11     0.0%
12     0.0%
13     0.0%
14     0.0%
15     0.0%
16     0.0%
17     0.0%
18     0.0%
19     0.0%
20     0.0%
21     0.0%
22     0.0%
23     0.0%
24     0.0%
25     0.0%
26     0.0%
27     0.0%
28     0.0%
29     0.0%
30     0.0%
31     0.0%
32     0.0%
Name: SellerNameSequenceNumber, dtype: object

In [185]:
SellerMailAddress = pd.read_csv(
    '../20221103_GA/ZTrans/SellerMailAddress.txt',
    sep='|',
    on_bad_lines='skip',
    encoding='latin-1',
    quoting=csv.QUOTE_NONE,
    header=None,
    usecols=[0, 1, 11, 12, 14, 15],
    names=["TransId", "SellerMailSequenceNumber", "SellerMailFullStreetAddress", "SellerMailCity", "SellerMailZip", "SellerMailZip4"],
    dtype={"TransId": "Int32", "SellerMailSequenceNumber": "Int32", "SellerMailFullStreetAddress": pd.StringDtype(),
           "SellerMailCity": "category", "SellerMailZip": "category", "SellerMailZip4": "category"})

In [186]:
SellerMailAddress

,TransId,SellerMailSequenceNumber,SellerMailFullStreetAddress,SellerMailCity,SellerMailZip,SellerMailZip4
0,148390483,1,<NA>,178 Torrance Rd,GA,31513
1,148390489,1,<NA>,160 Forest Ave,GA,31513
2,148390553,1,<NA>,25 Isaiah Ln,GA,31513
3,148390589,1,<NA>,1000 Vicars Landing Way,FL,32082
4,148390600,1,<NA>,199 Davis Landing Rd,GA,31513
...,...,...,...,...,...,...
399681,439721640,1,<NA>,8219 Trappers Creek Trl,VA,23832
399682,441087688,1,<NA>,348 Pecan St SW,GA,31781
399683,441087689,1,<NA>,424 Green St SW,GA,31781
399684,441087704,1,<NA>,3707 Hidden Hill Ct,GA,31721


In [187]:
SellerMailAddress['SellerMailSequenceNumber'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

1    100.0%
Name: SellerMailSequenceNumber, dtype: object

In [188]:
SellerName.fillna("NA", inplace=True)

In [189]:
SellerName['SellerNonIndividualName'] = SellerName['SellerNonIndividualName'].str.replace(',', '')

In [190]:
SellerName

,TransId,SellerIndividualFullName,SellerNonIndividualName,SellerNameSequenceNumber
0,148390440,DAVID ALLEN JOHNSON,NA,1
1,148390441,LENNOX M MORRIS,NA,1
2,148390442,JAMES MITCHELL WILLIAMS,NA,1
3,148390443,HERMAN MOBLEY,NA,1
4,148390445,NA,UNITED STATES DEPARTMENT O,1
...,...,...,...,...
5614666,578376772,PAMELA D COCHRAN,NA,1
5614667,578376773,BEN H HELTON,NA,1
5614668,578376774,AMANY T RODRIGUEZ,NA,1
5614669,578376775,CHARLENE N HOLLOWAY,NA,1


In [191]:
SellerNameAgg = SellerName.groupby("TransId", as_index=False)['SellerIndividualFullName', 'SellerNonIndividualName'].agg(','.join)
SellerNameAgg

C:\Users\NPOLIM~1\AppData\Local\Temp\93/ipykernel_140360/1014414994.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  SellerNameAgg = SellerName.groupby("TransId", as_index=False)['SellerIndividualFullName', 'SellerNonIndividualName'].agg(','.join)


,TransId,SellerIndividualFullName,SellerNonIndividualName
0,148390440,DAVID ALLEN JOHNSON,NA
1,148390441,LENNOX M MORRIS,NA
2,148390442,JAMES MITCHELL WILLIAMS,NA
3,148390443,HERMAN MOBLEY,NA
4,148390445,NA,UNITED STATES DEPARTMENT O
...,...,...,...
5338181,578458860,"TASHA FOLDS,TRACEY M FOLDS SR","NA,NA"
5338182,578458861,NA,JON'S CUSTOM HOMES INC
5338183,578458862,NA,MORRIS BANK
5338184,578458863,NA,BCB ENTITIES LLC


In [192]:
SellerNameAgg['SellerIndividualFullName1'] = SellerNameAgg[SellerNameAgg['SellerIndividualFullName'].str.contains(',')]['SellerIndividualFullName'].str.split(',').apply(lambda x: x[1])
SellerNameAgg

,TransId,SellerIndividualFullName,SellerNonIndividualName,SellerIndividualFullName1
0,148390440,DAVID ALLEN JOHNSON,NA,NaN
1,148390441,LENNOX M MORRIS,NA,NaN
2,148390442,JAMES MITCHELL WILLIAMS,NA,NaN
3,148390443,HERMAN MOBLEY,NA,NaN
4,148390445,NA,UNITED STATES DEPARTMENT O,NaN
...,...,...,...,...
5338181,578458860,"TASHA FOLDS,TRACEY M FOLDS SR","NA,NA",TRACEY M FOLDS SR
5338182,578458861,NA,JON'S CUSTOM HOMES INC,NaN
5338183,578458862,NA,MORRIS BANK,NaN
5338184,578458863,NA,BCB ENTITIES LLC,NaN


In [193]:
SellerNameAgg['SellerIndividualFullName'] = SellerNameAgg[SellerNameAgg['SellerIndividualFullName'].notna()]['SellerIndividualFullName'].str.split(',').apply(lambda x: x[0])
SellerNameAgg

,TransId,SellerIndividualFullName,SellerNonIndividualName,SellerIndividualFullName1
0,148390440,DAVID ALLEN JOHNSON,NA,NaN
1,148390441,LENNOX M MORRIS,NA,NaN
2,148390442,JAMES MITCHELL WILLIAMS,NA,NaN
3,148390443,HERMAN MOBLEY,NA,NaN
4,148390445,NA,UNITED STATES DEPARTMENT O,NaN
...,...,...,...,...
5338181,578458860,TASHA FOLDS,"NA,NA",TRACEY M FOLDS SR
5338182,578458861,NA,JON'S CUSTOM HOMES INC,NaN
5338183,578458862,NA,MORRIS BANK,NaN
5338184,578458863,NA,BCB ENTITIES LLC,NaN


In [194]:
SellerNameAgg['SellerNonIndividualName1'] = SellerNameAgg[SellerName['SellerNonIndividualName'].str.contains(',')]['SellerNonIndividualName'].str.split(',').apply(lambda x: x[1])
SellerNameAgg

C:\Users\NPOLIM~1\AppData\Local\Temp\93/ipykernel_140360/1016100767.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  SellerNameAgg['SellerNonIndividualName1'] = SellerNameAgg[SellerName['SellerNonIndividualName'].str.contains(',')]['SellerNonIndividualName'].str.split(',').apply(lambda x: x[1])


,TransId,SellerIndividualFullName,SellerNonIndividualName,SellerIndividualFullName1,SellerNonIndividualName1
0,148390440,DAVID ALLEN JOHNSON,NA,NaN,NaN
1,148390441,LENNOX M MORRIS,NA,NaN,NaN
2,148390442,JAMES MITCHELL WILLIAMS,NA,NaN,NaN
3,148390443,HERMAN MOBLEY,NA,NaN,NaN
4,148390445,NA,UNITED STATES DEPARTMENT O,NaN,NaN
...,...,...,...,...,...
5338181,578458860,TASHA FOLDS,"NA,NA",TRACEY M FOLDS SR,NaN
5338182,578458861,NA,JON'S CUSTOM HOMES INC,NaN,NaN
5338183,578458862,NA,MORRIS BANK,NaN,NaN
5338184,578458863,NA,BCB ENTITIES LLC,NaN,NaN


In [195]:
SellerNameAgg['SellerNonIndividualName'] = SellerNameAgg[SellerNameAgg['SellerNonIndividualName'].notna()]['SellerNonIndividualName'].str.split(',').apply(lambda x: x[0])
SellerNameAgg

,TransId,SellerIndividualFullName,SellerNonIndividualName,SellerIndividualFullName1,SellerNonIndividualName1
0,148390440,DAVID ALLEN JOHNSON,NA,NaN,NaN
1,148390441,LENNOX M MORRIS,NA,NaN,NaN
2,148390442,JAMES MITCHELL WILLIAMS,NA,NaN,NaN
3,148390443,HERMAN MOBLEY,NA,NaN,NaN
4,148390445,NA,UNITED STATES DEPARTMENT O,NaN,NaN
...,...,...,...,...,...
5338181,578458860,TASHA FOLDS,NA,TRACEY M FOLDS SR,NaN
5338182,578458861,NA,JON'S CUSTOM HOMES INC,NaN,NaN
5338183,578458862,NA,MORRIS BANK,NaN,NaN
5338184,578458863,NA,BCB ENTITIES LLC,NaN,NaN


In [196]:
SellerNameAgg.duplicated(subset='TransId').sum()

0

In [197]:
SellerMailAddress.duplicated(subset='TransId').sum()

0

In [198]:
sellerMerged = SellerName.merge(SellerMailAddress, how='left', on='TransId')

In [199]:
combined = combined.merge(buyerMerged, how='left', on='TransId')

In [200]:
combined = combined.merge(sellerMerged, how='left', on='TransId')

In [202]:
combined.columns

Index(['TransId', 'PropertyHouseNumber_x', 'PropertyFullStreetAddress_x',
       'PropertyCity_x', 'PropertyZip_x', 'PropertyZip4_x',
       'PropertySequenceNumber', 'FIPS_x', 'ImportParcelID', 'RowID', 'FIPS_y',
       'County_x', 'PropertyHouseNumber_y', 'PropertyFullStreetAddress_y',
       'PropertyCity_y', 'PropertyZip_y', 'PropertyZip4_y',
       'PropertyZoningSourceCode', 'CensusTract', 'TaxAmount', 'TaxYear',
       'TaxDelinquencyFlag', 'LotSizeSquareFeet', 'BatchID_x', 'NoOfUnits_x',
       'PropertyCountyLandUseCode_x', 'PropertyLandUseStndCode_x',
       'YearBuilt_x', 'NoOfStories_x', 'TotalRooms_x', 'TotalBedrooms_x',
       'TotalCalculatedBathCount_x', 'StoryTypeStndCode_x', 'FIPS', 'County_y',
       'DocumentTypeStndCode', 'RecordingDate', 'DataClassStndCode',
       'SalesPriceAmount', 'SalesPriceAmountStndCode',
       'IntraFamilyTransferFlag', 'PropertyUseStndCode', 'LoanTypeStndCode',
       'TotalDelinquentAmount', 'DelinquentAsOfDate', 'BatchID_y', 'FCDE',
  

In [204]:
print(len(combined[combined['PropertyHouseNumber_x'].isna()].index))
print(len(combined[combined['PropertyHouseNumber_y'].isna()].index))

389588
467911


In [205]:
combined['PropertyHouseNumber'] = combined['PropertyHouseNumber_x']
combined['PropertyHouseNumber'] = combined['PropertyHouseNumber'].fillna(combined['PropertyHouseNumber_y'])

In [207]:
print(len(combined[combined['PropertyHouseNumber'].isna()].index))

18684


In [209]:
combined = combined.drop(columns=['PropertyHouseNumber_x', 'PropertyHouseNumber_y'])
combined.columns

Index(['TransId', 'PropertyFullStreetAddress_x', 'PropertyCity_x',
       'PropertyZip_x', 'PropertyZip4_x', 'PropertySequenceNumber', 'FIPS_x',
       'ImportParcelID', 'RowID', 'FIPS_y', 'County_x',
       'PropertyFullStreetAddress_y', 'PropertyCity_y', 'PropertyZip_y',
       'PropertyZip4_y', 'PropertyZoningSourceCode', 'CensusTract',
       'TaxAmount', 'TaxYear', 'TaxDelinquencyFlag', 'LotSizeSquareFeet',
       'BatchID_x', 'NoOfUnits_x', 'PropertyCountyLandUseCode_x',
       'PropertyLandUseStndCode_x', 'YearBuilt_x', 'NoOfStories_x',
       'TotalRooms_x', 'TotalBedrooms_x', 'TotalCalculatedBathCount_x',
       'StoryTypeStndCode_x', 'FIPS', 'County_y', 'DocumentTypeStndCode',
       'RecordingDate', 'DataClassStndCode', 'SalesPriceAmount',
       'SalesPriceAmountStndCode', 'IntraFamilyTransferFlag',
       'PropertyUseStndCode', 'LoanTypeStndCode', 'TotalDelinquentAmount',
       'DelinquentAsOfDate', 'BatchID_y', 'FCDE', 'BuildingAreaStndCode',
       'BuildingAreaSqFt', '

### Step 10    
Save the combined dataframe to a file named 'ztrax_complete_ATL.csv'

In [201]:
.combined.to_csv('ztrax_FINAL_ATL.csv')

SyntaxError: invalid syntax (93/ipykernel_140360/2691758946.py, line 1)